# Q - Learning
---

* D.P 는 환경에 대한 확실한 확률정보를 알고있는 모델을 푸는 과정
* 환경에 불확실성이 포함된 모델을 푸는것이 RL 알고리즘
* 주로 sampling 기법으로 불확실성을 예측한다.
* 한꺼번에 모두 업데이트하는 D.P 와 다르게 한 step 씩 계산한다.
* Bellman Equation 을 변형하여 sampling 으로 얻은 data 를 적용한다.
* Off - Policy - 이 개념이 가장 중요
    * SARSA 에서 업그레이드된 알고리즘
    * 탐험을 할 수 있게 다른 분포를 가지는 policy 를 분리 -> 여기서 분리는 action 선택이지, 새로운 테이블은 없음
    * 원래 I.S 를 적용해야 하나, 단순히 policy 를 하나 더 만드는 것으로 간단하게 해결
    * SARSA 에서와 달리 명시적으로 다음 action 을 뽑지 않음. 그냥 즉시 최대 Q 가 되도록하는 action 일 때 Q 로 업데이트 함
    * 코드로는 1 줄도 안되는 이 내용이 실제로 꽤 중요한 이론이 녹아들어가 있음. 그리고 그 차이는 발견하기 어려울 수도 있음.
    * off policy 는 action 선택을 분리함으로써, q evaluation 을 더 객관적으로 바라볼 수 있음
    * SARSA 와는 엄연히 다름
* 학습률 alpha 는 학습 속도 조절에 중요한 파라미터
    * 수식 유도하면서 나온거지만 조절할 수 있음
    * 클수록 업데이트 양이 크고 작을수록 현재 Q 를 그대로 따르는 경향
* 한 스텝 업데이트 하는건, 테이블의 한 칸만 업데이트 하는 것
    * 그래서 저 업데이트 식이 이해가 됨. 이전 시점 state, 현재 시점 state 의 의미
* 차이점. 핵심
    * 위의 설명보다는 이 설명이 더 명료함. 다른 한국 블로그 글은 모두 엉터리, 틀린 설명인 듯하다
        * Q-Learning: Q(st+1,at+1) = max_a_Q(st+1,a)
        * SARSA: Q(st+1,at+1) = ε·mean_a_Q(st+1,a) + (1-ε)·maxaQ(st+1,a)
    * 단 하나의 차이점은, epsilon 의 유무일 뿐이다.
    * 만약 epsilon 이 0 이라면, SARSA 와 Q 러닝은 동일하다!
    * 즉, exploration 으로 인해 야기되는 차이에 기인함
    * a1, a2 뽑을 때 같은 epsilon greedy 를 적용할거나, 아니면 a2 뽑을 때는 완전 greedy 적용할거냐 차이
    * 그러니까 sarsa 는 exploration 을 2 번 하니까 더 두루두루 좋은 policy 를 갖는다.
    * 대신 q 러닝은 비교적 최적의 policy 를 학습할 수 있다.
    * cliff walking 예제에서 sarsa 가 더 안전한 길을 택하는데, exploration 을 많이 하면 더 최적의 길로 가야하지 않나?
    * 함정이 존재하는데, cliff 에 떨어지면 부정적 보상이 크기 때문에 평균적으로 더 점수가 높으려면 안전한 길로 가야함
    * 또한, epsilon 이 작아지면 점점 q 러닝과 유사해지다가 0 이 되면 q 러닝의 결과와 같아진다.
    * 즉, 결국엔 exploration 의 문제인 것임
    * 어떤 블로그에서 슈퍼마리오 예시 들 때 버섯을 밟고 다음 상황이 있었는데
        * 버섯 밟은 후 액션을 선택할 때 버섯 밟은 사실을 아는 여부가 on-off policy 차이점으로 이해하던데
        * 애초에 버섯 밟기는 둘 다 과거에 공통적으로 일어난 것이고, 차이점은 그 상황에서 얼마나 exploration 을 할거냐이다
        * sarsa 와 q 러닝의 차이점을 논할 때 exploration 을 빼놓으면 아무것도 없다.

In [1]:
import gym
import numpy as np
import random
from gym.envs.registration import register

In [2]:
'''
환경셋팅 한 후에 환경을 추가등록한다.
'''

register(
    id='FrozenLake-v1',
    entry_point="gym.envs.toy_text:FrozenLakeEnv",
    kwargs={'map_name':'4x4','is_slippery':False})

In [3]:
'''
환경 생성
'''
env = gym.make('FrozenLake-v1')

In [8]:
q_table = np.zeros([env.action_space.n, env.observation_space.n], dtype = np.float16)
gamma = .9
epsilon = 1
episode = 0
max_episode = 1000

state = env.reset()
step = 0

while(episode < max_episode):
    
    #env.render()
    step += 1
    #reward -= step*.01
        
    if(random.random() > epsilon):
        action = np.argmax(q_table[ : , state_next])
    else:
        action = env.action_space.sample()
    
    state_next, reward, done, _ = env.step(action)

    # Q - Learning
    q_table[action, state] += gamma * (reward + np.max(q_table[ : , state_next]) - q_table[action, state])
    
    state_old = state    
    state = state_next
    
    if(done):
        if(reward):
            if(epsilon > .1):
                epsilon = 1 / (episode/(max_episode/10) + 1)
            else:
                epsilon = .1
        else:
            q_table[action, state_old] += gamma * (-1 + q_table[action, state] - q_table[action, state_old])
        
        step = 0
        episode += 1
        env.reset()
env.close()

In [9]:
q_table

array([[ 95.1   , 106.6   ,  17.27  ,   0.    ,  93.9   ,  53.5   ,
         -0.909 ,   0.    , 107.75  ,  98.5   , 105.    ,  11.42  ,
         24.47  ,  30.84  , 107.2   ,  83.75  ],
       [111.5   ,   5.875 ,   0.    ,  -0.909 , 112.1   ,  96.44  ,
          0.729 ,   0.    ,  84.1   , 112.4   ,  85.5   ,   0.    ,
        105.75  ,  98.75  , 103.2   , 111.8   ],
       [ 74.94  ,  12.14  ,   0.    ,   0.    ,  58.7   ,  55.62  ,
         -0.909 ,   0.    , 112.4   , 102.44  ,  -0.909 ,   0.    ,
         67.7   , 112.4   , 112.75  ,  97.    ],
       [ 70.1   ,   5.28  ,   0.    ,   0.    ,  96.75  , 100.6   ,
          0.    ,   0.    ,  82.25  ,  99.6   ,   0.6562,   0.    ,
         22.44  , 105.2   ,  97.4   , 109.8   ]], dtype=float16)

In [6]:
epsilon

0.1

In [10]:
s = env.reset()

step = 1
while(True):
    env.render()
    a = np.argmax(q_table[ : , s])
    s,r,d,_ = env.step(a)
    if(d):
        env.render()
        break
env.close()


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
